In [2]:
import pandas as pd

In [3]:
babylon = pd.read_csv('babylonbee_urls.csv', quotechar="\"")

In [4]:
onion = pd.read_csv('onion_urls_brief.csv')

In [5]:
onion = onion.drop_duplicates()

In [6]:
spoof_2020 = pd.read_csv('spoof_urls_2020.csv')

In [7]:
spoof_2021 = pd.read_csv('spoof_urls_2021.csv')

In [8]:
satire = pd.concat([babylon, onion, spoof_2020, spoof_2021])

In [9]:
wel = pd.read_csv('../data/WELFake_Dataset.csv')

In [10]:
wel = wel.dropna()

In [11]:
wel[wel['text'].str.contains('Reuters')]

,Unnamed: 0,title,text,label
11,11,"May Brexit offer would hurt, cost EU citizens ...",BRUSSELS (Reuters) - British Prime Minister Th...,0
12,12,Schumer calls on Trump to appoint official to ...,"WASHINGTON (Reuters) - Charles Schumer, the to...",0
15,15,Billionaire Odebrecht in Brazil scandal releas...,RIO DE JANEIRO/SAO PAULO (Reuters) - Billionai...,0
17,17,U.N. seeks humanitarian pause in Sanaa where s...,GENEVA (Reuters) - The United Nations called o...,0
19,19,Second judge says Clinton email setup may have...,NEW YORK (Reuters) - A second federal judge ha...,0
...,...,...,...,...
72112,72112,United States and Cuba complete deals as Trump...,HAVANA (Reuters) - The Obama administration an...,0
72120,72120,North Korea's Kim congratulates China's Xi aft...,SEOUL (Reuters) - North Korean leader Kim Jong...,0
72122,72122,Swiss stop seizing income from asylum seekers ...,ZURICH (Reuters) - Switzerland will stop seizi...,0
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0


In [12]:
fake_wel = wel[wel['label'] == 1]
real_wel = wel[wel['label'] == 0]

In [13]:
small_real = real_wel.sample(n=12500, random_state=1123456)
small_fake = fake_wel.sample(n=12500, random_state=1123456)

In [14]:
satire['label'] = 'satire'

In [15]:
small_real['label'] = 'real'

In [16]:
small_fake['label'] = 'fake'

In [17]:
small_real.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)

In [18]:
small_fake.rename(columns={'title' : 'Heading', 'text' : 'Body'}, inplace=True)

In [19]:
satire_final = satire[['Heading', 'Body', 'label']]
fake_final = small_fake[['Heading', 'Body', 'label']]
real_final = small_real[['Heading', 'Body', 'label']]

In [20]:
final_data = pd.concat([satire_final, fake_final, real_final])

In [21]:
final_data['Heading_Body'] = final_data[['Heading', 'Body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [22]:
final_data[['Heading', 'Body', 'label']].to_csv('final_data_separate.csv', index=False)
final_data[['Heading_Body', 'label']].to_csv('final_data_combined.csv', index=False)

In [23]:
from collections import Counter
# Create Vocabulary
counter = Counter()
for comment in final_data.Heading_Body:
    counter.update(comment.split())

In [41]:
clean_final_data = final_data[['Heading_Body', 'label']]


In [42]:
clean_final_data['Heading_Body']

0        Here Are 10 Babylon Bee Jokes Explained (Just ...
1        Pete Buttigieg Says He Cannot Come Into Work A...
2        Instead Of Kryptonite, New LGBTQ+ Superman Wil...
3        Bernie Retires As His Vision Of Making The U.S...
4        Katie Couric Admits She Edited Interview To Re...
                               ...                        
46449    British ministers rally around Foreign Secreta...
8765     Mexicans on U.S. border fear economic catastro...
23563    Let’s dispel with this notion that Donald Trum...
66773    Explosions rock Myanmar area near Bangladesh b...
61764    Comedy Central Launches Late-Night Trump-Mocki...
Name: Heading_Body, Length: 37504, dtype: object

In [51]:
import re
# Convert to lowercase
clean_final_data = clean_final_data.applymap(lambda s: s.lower())
# Fix Contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)
#clean_final_data.loc['Heading_Body'] = clean_final_data['Heading_Body'].apply(replace_contractions)

import string
# Remove Puncuation
def remove_punc(s):
    table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
    new_s = s.translate(table)

clean_final_data = clean_final_data.apply(lambda s: s.translate(str.maketrans('', '', string.punctuation)), axis=1)
# Remove references to Babylon Bee, The Onion, Satire


AttributeError: ("'Series' object has no attribute 'translate'", 'occurred at index 0')

In [47]:
lower_case.values

array(['here are 10 babylon bee jokes explained (just in case you don\'t get them) from the atlantic to the pacific a lot of people all across the country don\'t understand babylon bee jokes. that\'s because we conservative christians are bad at humor. we\'re nowhere near as funny as leftist humorists like samantha bee and jimmy kimmel.\xa0so to make sure you understand the headlines at the babylon bee we\'ve included ten classic bee articles here along with a helpful explanation.ok here we go:explanation: the joke here is that aoc (who is dumb) accidentally strangled herself tying her shoes which is a simple task for most humans\xa0because she is so stupid. hope this is helpful.explanation: the joke here is that democrats called for flags to be flown at half-mast to grieve the death of general soleimani. get it? a terrorist died and democrats were sad.explanation: this one is a joke that nba players were wearing special lace collars to honor ruth bader ginsburg who had recently passed